In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from ),
    end_time=(fetch_data_to),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-05 00:21:12,371 INFO: Initializing external client
2025-03-05 00:21:12,372 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 00:21:13,106 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215658
Fetching data from 2025-02-04 05:21:12.369953+00:00 to 2025-03-05 04:21:12.369953+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (30.07s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-04 06:00:00+00:00,2,0
1,2025-02-04 07:00:00+00:00,2,0
2,2025-02-04 08:00:00+00:00,2,0
3,2025-02-04 09:00:00+00:00,2,0
4,2025-02-04 10:00:00+00:00,2,0
...,...,...,...
174440,2025-03-05 00:00:00+00:00,263,22
174441,2025-03-05 01:00:00+00:00,263,10
174442,2025-03-05 02:00:00+00:00,263,5
174443,2025-03-05 03:00:00+00:00,263,3


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174445 entries, 0 to 174444
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174445 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174445 non-null  int32                  
 2   rides               174445 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 2.7 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174445 entries, 0 to 174444
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174445 non-null  datetime64[us]
 1   pickup_location_id  174445 non-null  int32         
 2   rides               174445 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 2.7 MB


In [9]:
ts_data = ts_data.rename(columns={"value": "rides"})

In [10]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [11]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,0,0,0,0,3,0,0,3,5,...,0,0,3,0,1,0,1,0,255,2025-02-12 17:00:00
1,0,0,1,0,0,0,6,2,0,0,...,2,0,1,0,0,1,0,1,112,2025-03-04 09:00:00
2,2,1,0,0,1,1,0,1,0,1,...,0,0,0,0,1,0,0,2,218,2025-02-11 16:00:00
3,369,273,249,21,251,348,17,373,286,251,...,275,1,222,32,324,393,166,35,237,2025-02-20 23:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,57,2025-02-04 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,1,0,...,1,0,0,1,0,0,1,0,108,2025-02-23 15:00:00
247,0,0,0,2,0,0,4,0,1,0,...,1,0,3,2,0,0,0,2,71,2025-02-08 05:00:00
248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,2025-02-26 16:00:00
249,0,1,1,1,1,1,2,0,4,0,...,1,1,2,4,1,0,0,0,168,2025-02-09 06:00:00


In [12]:
# Ensure `pickup_hour` is timezone-aware in UTC
ts_data["pickup_hour"] = pd.to_datetime(ts_data["pickup_hour"], utc=True)

# Ensure `fetch_data_from` and `fetch_data_to` are also timezone-aware
fetch_data_from = fetch_data_from.tz_convert("UTC") if fetch_data_from.tz else fetch_data_from.tz_localize("UTC")
fetch_data_to = fetch_data_to.tz_convert("UTC") if fetch_data_to.tz else fetch_data_to.tz_localize("UTC")

# Apply filtering
filtered_ts_data = ts_data[ts_data["pickup_hour"].between(fetch_data_from, fetch_data_to)]

print("Filtered ts_data:")
print(filtered_ts_data.head())

Filtered ts_data:
                pickup_hour  pickup_location_id  rides
0 2025-02-19 19:00:00+00:00                 255      2
1 2025-03-04 00:00:00+00:00                 112      0
2 2025-02-23 08:00:00+00:00                 218      2
3 2025-02-13 10:00:00+00:00                 237    369
4 2025-02-28 14:00:00+00:00                  57      0


In [13]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-05 00:23:20,417 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 00:23:20,423 INFO: Initializing external client
2025-03-05 00:23:20,423 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 00:23:20,921 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215658
Fetching data from 2025-02-04 05:23:20.417699+00:00 to 2025-03-05 04:23:20.417699+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.96s) 


In [14]:
current_date

Timestamp('2025-03-05 05:23:20.417699+0000', tz='Etc/UTC')

In [15]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-04 06:00:00
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-04 06:00:00
2,2,1,11,7,3,2,3,2,1,7,...,1,1,4,4,3,0,0,1,4,2025-03-04 06:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-04 06:00:00
4,2,2,1,2,3,2,3,1,2,4,...,1,0,0,1,0,0,0,0,7,2025-03-04 06:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,259,2025-03-04 06:00:00
247,1,4,2,0,1,1,1,0,1,1,...,0,1,0,1,0,0,0,1,260,2025-03-04 06:00:00
248,0,4,5,16,21,34,25,31,37,28,...,8,10,1,0,0,1,1,1,261,2025-03-04 06:00:00
249,63,126,177,156,102,81,88,81,79,97,...,14,9,2,2,1,4,4,16,262,2025-03-04 06:00:00


In [17]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 00:23:50,838 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 00:23:50,840 INFO: Initializing external client
2025-03-05 00:23:50,840 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 00:23:51,401 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215658


In [18]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [19]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,1.0
3,6,0.0
4,7,2.0
...,...,...
246,259,1.0
247,260,1.0
248,261,1.0
249,262,68.0


In [20]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 186, 100, 162, 262,  48, 239, 236, 237, 238], dtype=int32)